# Quarterly Exchange Rate Analysis

## Description
This notebook processes exchange rate data. It
- **Reads**: CSV files containing daily exchange rates for Chinese Yuan to USD (`Chinese yuan to USD.csv`), South Korean Won to USD (`South Korean WON to USD (Exchange rate).csv`), and Japanese Yen to USD (`Japanese Yen to USD (Exchange Rate).csv`).
- **Processes**: Standardizes column names, converts dates, calculates quarterly mean exchange rates, and merges results.
- **Produces**: A CSV file (`exchange_rate_ivs.csv`) with quarterly mean exchange rates for the three currencies, indexed by quarter (e.g., '2022Q1').


In [ ]:
import pandas as pd
import numpy as np

# ------------------------------
# Data Loading and Initial Cleaning
# ------------------------------
# Load exchange rate data for Chinese Yuan, South Korean Won, and Japanese Yen to USD.
yuan_usd = pd.read_csv("/content/drive/MyDrive/PhD thesis (data files)/Chinese yuan to USD.csv")
won_usd = pd.read_csv("/content/drive/MyDrive/PhD thesis (data files)/South Korean WON to USD (Exchange rate).csv")
yen_usd = pd.read_csv("/content/drive/MyDrive/PhD thesis (data files)/Japanese Yen to USD (Exchange Rate).csv")

# Standardize column names for consistency across DataFrames.
won_usd = won_usd.rename(columns={"DEXKOUS": "value"})
yen_usd = yen_usd.rename(columns={"DEXJPUS": "value"})

# Convert 'DATE' columns to datetime and drop original 'DATE' column.
dataframes = [yuan_usd, won_usd, yen_usd]
for df in dataframes:
    df["date"] = pd.to_datetime(df["DATE"])
    df.drop(columns=["DATE"], inplace=True)

# Convert 'value' columns to numeric, coercing invalid entries to NaN.
for df in [won_usd, yen_usd]:
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

## Extract Month and Year

Extract `month` and `year` from the `date` column to facilitate quarterly grouping. This uses pandas' vectorized datetime operations for efficiency.

In [ ]:
# Add 'month' and 'year' columns for quarter calculation.
for df in dataframes:
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year

## Calculate Quarterly Means

Define a function to calculate quarterly means for exchange rates. The function:
- Assigns quarters (Q1, Q2, Q3, Q4) based on months.
- Creates a `quarter` column (e.g., '2022Q1').
- Returns a DataFrame with the mean exchange rate per quarter.

In [ ]:
def calculate_quarterly_means(df):
    """
    Calculate the quarterly mean of exchange rate values from a DataFrame.

    Parameters:
        df (pd.DataFrame): Input DataFrame with 'date', 'value', 'month', and 'year' columns.

    Returns:
        pd.DataFrame: DataFrame with quarterly mean values, indexed by 'quarter' (e.g., '2022Q1').
    """
    # Assign quarters using vectorized operations for efficiency.
    conditions = [
        df["month"].isin([1, 2, 3]),
        df["month"].isin([4, 5, 6]),
        df["month"].isin([7, 8, 9]),
        df["month"].isin([10, 11, 12])
    ]
    choices = ["Q1", "Q2", "Q3", "Q4"]
    df["quarter"] = np.select(conditions, choices)
    
    # Create 'quarter' column by concatenating year and quarter.
    df["quarter"] = df["year"].astype(str) + df["quarter"]
    
    # Group by quarter and calculate mean of 'value'.
    return df.groupby("quarter")["value"].mean().reset_index()

## Apply Quarterly Calculation

Apply the `calculate_quarterly_means` function to each currency's DataFrame to compute quarterly averages.

In [ ]:
# Calculate quarterly means for each currency.
yuan_quarterly = calculate_quarterly_means(yuan_usd)
won_quarterly = calculate_quarterly_means(won_usd)
yen_quarterly = calculate_quarterly_means(yen_usd)

## Merge and Save Results

Merge the quarterly DataFrames on the `quarter` column, rename columns for clarity, and save the result to a CSV file.

In [ ]:
# Merge DataFrames on 'quarter' column.
merged_df = pd.merge(yen_quarterly, won_quarterly, on="quarter", suffixes=("_yen", "_won"))
merged_df = pd.merge(merged_df, yuan_quarterly, on="quarter")

# Rename columns for clarity and consistency.
merged_df.columns = ["quarter", "japan_usd", "korea_usd", "china_usd"]

# Save the merged DataFrame to a CSV file.
merged_df.to_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/exchange_rate_ivs.csv", index=False)

The code in in file: "final merge(with_ivs)" is infact implemented below because it was too short for a separate file

In [ ]:
#loading data
phone_arena=pd.read_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/Phone Arena data IDC(merged).csv")

#merging with phone arena
final_df=pd.merge(phone_arena,merged_df,on="Quarter",how="left")

#converting to csv
final_df.to_csv("/content/drive/MyDrive/OUTPUT DATA(FROM IDC)/complete data( exchange rates IDC, Phonearena(Feb 9,2024).csv")
